In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
from shapely.prepared import prep
from bokeh.models import ColumnDataSource, Patches, HoverTool, TapTool, Plot, Range1d
from bokeh.plotting import figure, show, output_notebook
output_notebook()

BokehJS successfully loaded.

In [2]:
## Read from json and convert to np arrays
map_data = GeoDataFrame.from_file('Neighborhood_Clusters.geojson')
map_data.head()

,NAME,NBH_NAMES,OBJECTID,SHAPE_Area,SHAPE_Length,WEB_URL,geometry
0,Cluster 39,"Congress Heights, Bellevue, Washington Highlands",1,4887587.842320,10719.614725,http://planning.dc.gov/,POLYGON ((-77.01389426988646 38.82865056750391...
1,Cluster 38,"Douglas, Shipley Terrace",2,2370929.441790,8233.450913,http://planning.dc.gov/,POLYGON ((-76.99291178387195 38.85390625018695...
2,Cluster 36,"Woodland/Fort Stanton, Garfield Heights, Knox ...",3,1119573.063962,4746.344538,http://planning.dc.gov/,POLYGON ((-76.97714019966327 38.85727998162274...
3,Cluster 27,"Near Southeast, Navy Yard",4,1619166.846789,7286.968935,http://planning.dc.gov/,POLYGON ((-76.97936283229114 38.87755461953095...
4,Cluster 32,"River Terrace, Benning, Greenway, Dupont Park",5,4286976.980060,11258.342225,http://planning.dc.gov/,POLYGON ((-76.93760147029884 38.88995958845386...


Load in NFIRS and CAD data for community.

In [3]:
data = pd.read_csv('DC_CAD_FY1415.csv') # read in CAD data
nfirs = pd.read_csv('DC_NFIRS_FY1415.csv') # read in NFIRS data
merged_data = pd.merge(left=data,right=nfirs, left_on='num_1', right_on='num_1') # merge nfirs and cad based on incident number
data = merged_data[pd.to_numeric(merged_data['incidenttype'],errors='coerce')<130] # select only structure fires
data = data[data['num_1'].str.contains("F14")] #sliced year data

Take geospatial fire data and place it appropriate polygon.

In [4]:
mapped_points = GeoSeries([Point(mapped_x, mapped_y) for mapped_x, mapped_y in zip(data['longitude'],data['latitude'])])
all_points = MultiPoint(mapped_points)
all_polygons = prep(MultiPolygon(list(map_data['geometry'].values)))
city_points = list(filter(all_polygons.contains, all_points))
def num_of_contained_points(apolygon, city_points):
    return int(len(list(filter(prep(apolygon).contains, city_points))))

map_data['hood_count'] = map_data['geometry'].apply(num_of_contained_points, args=(city_points,)) #counts number of fires per neighborhood

Color bins based on number of fires in each bin.

In [5]:
color_split = int(max(map_data['hood_count'])/5)
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]
hood_colors =[]
for i in range(len(map_data['NBH_NAMES'])):
    idx = min(int(np.around(map_data.iloc[i]['hood_count']/color_split)),5)
    hood_colors.append(colors[idx])
map_data['hood_colors']=hood_colors

Write appended data file out to new geojson file.

In [6]:
map_data.to_file('DC_Fires.geojson', driver="GeoJSON")

Check to  make sure new file loads and looks correct.

In [7]:
map_data2 = GeoDataFrame.from_file('DC_Fires.geojson')
map_data2.head()

,NAME,NBH_NAMES,OBJECTID,SHAPE_Area,SHAPE_Length,WEB_URL,geometry,hood_colors,hood_count
0,Cluster 39,"Congress Heights, Bellevue, Washington Highlands",1,4887587.842320,10719.614725,http://planning.dc.gov/,POLYGON ((-77.01389426988646 38.82865056750391...,#980043,84
1,Cluster 38,"Douglas, Shipley Terrace",2,2370929.441790,8233.450913,http://planning.dc.gov/,POLYGON ((-76.99291178387195 38.85390625018695...,#DF65B0,57
2,Cluster 36,"Woodland/Fort Stanton, Garfield Heights, Knox ...",3,1119573.063962,4746.344538,http://planning.dc.gov/,POLYGON ((-76.97714019966327 38.85727998162274...,#C994C7,28
3,Cluster 27,"Near Southeast, Navy Yard",4,1619166.846789,7286.968935,http://planning.dc.gov/,POLYGON ((-76.97936283229114 38.87755461953095...,#F1EEF6,7
4,Cluster 32,"River Terrace, Benning, Greenway, Dupont Park",5,4286976.980060,11258.342225,http://planning.dc.gov/,POLYGON ((-76.93760147029884 38.88995958845386...,#C994C7,32
